In [1]:
import logging
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import os
from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from utils.data_loading import SatelliteDataset
from utils.dice_score import dice_loss
from evaluate import evaluate
from unet import SiamUNetConCResnet50,vgg19nobn_unetdouble,VGG16Unet,resnet_unet,resnet_siamunet,SiamUNetDiffVgg19Space, SiamUNetFullConCVgg19,SiamUNetDiffVgg19,UNETResnet50,UNet,SiamUNet,SiamUNetConC,SiamUNetDiff, SiamUNetConCVgg19, VGG19Unet,vgg19bn_unet,vgg19nobn_unet,vgg16bn_unet, UNetWithResnet50Encoder, UNetWithVgg19BnEncoder, SiameseUNetWithResnet50Encoder, SiamUnet_diff_Full, SiameseUNetV2
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses.focal import FocalLoss
import numpy as np
from torchmetrics import ConfusionMatrix

C:\Users\thanh\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\thanh\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


None
C:\Users\thanh/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
None
C:\Users\thanh/.cache\torch\hub\checkpoints\vgg19_bn-c79401a0.pth


C:\Users\thanh\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
post_dir_img = Path('.\\Dataset\\TierFull\\Post\\Image512\\')
post_dir_mask = Path('.\\Dataset\\TierFull\\Post\\Label512\\')
pre_dir_img = Path('.\\Dataset\\TierFull\\Pre\\Image512\\')
pre_dir_mask = Path('.\\Dataset\\TierFull\\Pre\\Label512\\')
post_dir_val_img = Path('.\\Dataset\\Validation\\Post\\Image512\\')
post_dir_val_mask = Path('.\\Dataset\\Validation\\Post\\Label512\\')
pre_dir_val_img = Path('.\\Dataset\\Validation\\Pre\\Image512\\')
pre_dir_val_mask = Path('.\\Dataset\\Validation\\Pre\\Label512\\')
dir_checkpoint = Path('./checkpoints/Vgg19SiamConc/')

In [3]:
closs = nn.CrossEntropyLoss()

floss = FocalLoss(mode = 'multiclass',
                alpha = None,
                gamma = 2.0,
                ignore_index = None,
                reduction = "mean",
                normalized = False,
                reduced_threshold = None)

In [4]:
def get_class_weights():
    train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[1,1], True, True, 0, None, 0, 0], probabilities = [.5,.5,.5,0,0,0,0],increase = 8397,mask_suffix = '.png')
    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
    train_loader = DataLoader(train, shuffle=True, **loader_args)
    classes = torch.zeros(5)
    with tqdm(total=len(train_loader), unit='img') as pbar:
        for batch in train_loader:
            image = batch['mask'].int()
            count = torch.bincount(torch.flatten(image),minlength = 5)
            classes = classes.add(count)
            pbar.update(image.shape[0])
    return torch.div(classes,torch.sum(classes))

In [5]:
def train_net(net,
              device,
              start_epoch: int = 1,
              epochs: int = 5,
              batch_size: int = 1,
              learning_rate: float = 1e-5,
              train_percent: float = 0.5,
              val_percent: float = 0.1,
              save_checkpoint: bool = True,
              img_scale: float = 0.5,
              ampbool: bool = False,
              traintype: str = 'post',
              gradient_clipping: float = 1.0):
    # 1. Create dataset
    try:
        train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0],increase = 0 ,mask_suffix = '.png',normalizemodel = 'vgg19')
        validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0],increase = 0,mask_suffix = '.png', normalizemodel = 'vgg19')
    except (AssertionError, RuntimeError):
        print('error')

    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
        
    # 2. Split into train / validation partitions
    n_train = int(len(train) * train_percent)
    n_train_none = int(len(train) - n_train)
    n_val = int(len(validate) * val_percent)
    n_val_none = int(len(validate) - n_val)

    train_set, train_val_none_set = random_split(train, [n_train, n_train_none], generator=torch.Generator().manual_seed(0))
    
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)    

    val_set, val_none_set = random_split(validate, [n_val, n_val_none], generator=torch.Generator().manual_seed(0))    

    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)    

    # 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
    #optimizer = optim.AdamW(net.parameters(), lr=learning_rate, weight_decay=1e-6)
    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, weight_decay=1e-6, momentum=0.9, foreach=True)
    #scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 5, 1,eta_min = learning_rate/100)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,6,9,12,15,18,19,20, 33, 47, 50, 60, 70, 90, 110, 130, 150, 170, 180, 190], gamma=0.5)
    grad_scaler = torch.cuda.amp.GradScaler(enabled=ampbool)
    criterion = closs
    global_step = 0

    # 5. Begin training
    for epoch in range(start_epoch, start_epoch + epochs):
        net.train()
        epoch_loss = 0
        nancount = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                optimizer.zero_grad(set_to_none=True)
                preimage, postimage, post_masks, pre_masks = batch['preimage'], batch['image'], batch['mask'], batch['premask']

                preimage = preimage.to(device=device, dtype=torch.float32)
                postimage = postimage.to(device=device, dtype=torch.float32)
                post_masks = post_masks.to(device=device, dtype=torch.long)
                pre_masks = pre_masks.to(device=device, dtype=torch.long)
                with torch.cuda.amp.autocast(enabled = ampbool):
                    masks_pred = None
                    if(traintype == 'both'):
                        masks_pred = net(preimage,postimage)
                        loss = criterion(masks_pred, post_masks)
                        loss += dice_loss(
                            F.softmax(masks_pred, dim=1).float()[:, 1:, ...],
                            F.one_hot(post_masks, 5).permute(0, 3, 1, 2).float()[:, 1:, ...],
                            multiclass=True
                        )
                    if(traintype == 'pre'):
                        masks_pred = net(preimage)
                        loss = criterion(masks_pred, pre_masks)
                        loss += dice_loss(
                            F.softmax(masks_pred, dim=1).float(),
                            F.one_hot(pre_masks, 2).permute(0, 3, 1, 2).float(),
                            multiclass=True
                        )
                    if(traintype == 'post'):
                        masks_pred = net(postimage)
                        loss = criterion(masks_pred, post_masks)
                        loss += dice_loss(
                            F.softmax(masks_pred, dim=1).float()[:, 1:, ...],
                            F.one_hot(post_masks, 5).permute(0, 3, 1, 2).float()[:, 1:, ...],
                            multiclass=True
                        )
                    
                grad_scaler.scale(loss).backward()
                #torch.nn.utils.clip_grad_norm_(net.parameters(), gradient_clipping)
                grad_scaler.step(optimizer)
                grad_scaler.update()
                pbar.update(postimage.shape[0])
                global_step += 1
                if(math.isnan(loss.item())):
                    epoch_loss+=0
                    nancount +=1
                else:
                    epoch_loss += loss.item()
                pbar.set_postfix(**{'loss (batch)': loss.item()})
                        
        val_score, val_classes,val_loss = evaluate(net,dataloader = val_loader,device = device,ampbool = ampbool,traintype = traintype)
        scheduler.step(val_score)
        print(val_score)
        print(val_classes)
        print(val_loss)
        print(epoch_loss/n_train)
        print(nancount)
        if save_checkpoint:
            Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
            torch.save(net.state_dict(), str(dir_checkpoint / 'branch12checkpoint_epoch_branch_{}.pth'.format(epoch)))
            logging.info(f'Checkpoint {epoch} saved!')

In [6]:
classes = 5
bilinear = True
loadstate = True
load = './checkpoints/Vgg19SiamConc/checkpoint_epoch12.pth'
start_epoch = 13
epochs = 20
batch_size = 1
lr = 1e-6
scale = 1
train =0.15259598603*2
val = .50
ampbool = True
save_checkpoint = True
traintype = 'both'
gradclip = 1.0

In [7]:
def confusionmatrix(pred,true):
    result = np.zeros((5,5))
    for i in range(true.shape[1]):
        for j in range(true.shape[2]):
            result[true[0][i][j]][pred[0][i][j]] +=1
    return result

In [8]:
def modelconfusionmatrix(filepath,net,train_percent,val_percent):
    device = 'cuda'
    net.load_state_dict(torch.load(filepath, map_location=device))
    net.to(device=device)
    net.eval()
    try:
        train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0],increase = 0 ,mask_suffix = '.png',normalizemodel = 'vgg19')
        validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0],increase = 0,mask_suffix = '.png', normalizemodel = 'vgg19')
    except (AssertionError, RuntimeError):
        print('error')

    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
        
    # 2. Split into train / validation partitions
    n_train = int(len(train) * train_percent)
    n_train_none = int(len(train) - n_train)
    n_val = int(len(validate) * val_percent)
    n_val_none = int(len(validate) - n_val)

    train_set, train_val_none_set = random_split(train, [n_train, n_train_none], generator=torch.Generator().manual_seed(0))
    
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)    

    val_set, val_none_set = random_split(validate, [n_val, n_val_none], generator=torch.Generator().manual_seed(0))    

    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args) 
    resulttrain = torch.zeros(5,5).to(device)
    resultval = torch.zeros(5,5).to(device)

    num_val_batches = len(val_loader)
    num_train_batches = len(train_loader)
    confmat = ConfusionMatrix(task="multiclass", num_classes=5).to(device)

    with tqdm(total=num_train_batches, desc='train', unit='img') as pbar:
        for batch in train_loader:
            preimage, postimage, true_masks = batch['preimage'], batch['image'], batch['mask']
            preimage = preimage.to(device=device, dtype=torch.float32)
            postimage = postimage.to(device=device, dtype=torch.float32)
            true_masks = true_masks.to(device=device, dtype=torch.long)
            mask_true = F.one_hot(true_masks, 5).permute(0,3, 1, 2).float()
            with torch.cuda.amp.autocast(enabled = True):
                # predict the mask
                mask_pred = net(preimage,postimage)
                # convert to one-hot format
                pred = F.softmax(mask_pred, dim=1).int().argmax(-3)
                true = F.one_hot(true_masks, 5).int().permute(0, 3, 1, 2).argmax(-3)
                resulttrain += confmat(pred,true)
            pbar.update(postimage.shape[0])
    print(resulttrain)
    with tqdm(total=num_val_batches, desc='validation', unit='img') as pbar:
        for batch in val_loader:
            preimage, postimage, true_masks = batch['preimage'], batch['image'], batch['mask']
            preimage = preimage.to(device=device, dtype=torch.float32)
            postimage = postimage.to(device=device, dtype=torch.float32)
            true_masks = true_masks.to(device=device, dtype=torch.long)
            mask_true = F.one_hot(true_masks, 5).permute(0,3, 1, 2).float()
            with torch.cuda.amp.autocast(enabled = True):
                # predict the mask
                mask_pred = net(preimage,postimage)
                # convert to one-hot format
                pred = F.softmax(mask_pred, dim=1).int().argmax(-3)
                true = F.one_hot(true_masks, 5).int().permute(0, 3, 1, 2).argmax(-3)
                resultval += confmat(pred,true)
            pbar.update(postimage.shape[0])
    print(resultval)
    net.train()

    # Fixes a potential division by zero error
    if num_val_batches == 0:
        return resulttrain,resultval
    return resulttrain,resultval

In [9]:
def EvaluateFolder(folderpath,net,train_percent,val_percent):
    in_files = [folderpath + s for s in os.listdir(folderpath)]
    try:
        train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0],increase = 0 ,mask_suffix = '.png',normalizemodel = 'vgg19')
        validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0],increase = 0,mask_suffix = '.png', normalizemodel = 'vgg19')
    except (AssertionError, RuntimeError):
        print('error')

    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
        
    # 2. Split into train / validation partitions
    n_train = int(len(train) * train_percent)
    n_train_none = int(len(train) - n_train)
    n_val = int(len(validate) * val_percent)
    n_val_none = int(len(validate) - n_val)

    train_set, train_val_none_set = random_split(train, [n_train, n_train_none], generator=torch.Generator().manual_seed(0))
    
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)    

    val_set, val_none_set = random_split(validate, [n_val, n_val_none], generator=torch.Generator().manual_seed(0))    

    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)
    
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    
    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
    
    for i in in_files:
        net.load_state_dict(torch.load(i, map_location=device))
        net.to(device=device)
        #val_score, val_classes,val_loss = evaluate(net,dataloader = val_loader,device = device,ampbool = True,traintype = 'both')
        train_score, train_classes,train_loss = evaluate(net,dataloader = train_loader,device = device,ampbool = True,traintype = 'both')
        print('Train: ' + i)
        print(train_score)
        print(train_loss)
        #print('Validation: ' + i)
        #print(val_score)
        #print(val_loss)

In [10]:
net = SiamUNetConCVgg19()


None
C:\Users\thanh/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth


In [11]:
EvaluateFolder('./checkpoints/Vgg19SiamConc/',net,train,val)

validation: 100%|█████████████████████████████████████████████████| 2797/2797 [06:04<00:00,  7.67img/s, loss (batch)=1]


Train: ./checkpoints/Vgg19SiamConc/checkpoint_epoch1.pth
tensor(0.7376, device='cuda:0')
1.0003046989440918


validation: 100%|█████████████████████████████████████████████| 2797/2797 [06:02<00:00,  7.72img/s, loss (batch)=0.915]


Train: ./checkpoints/Vgg19SiamConc/checkpoint_epoch10.pth
tensor(0.7143, device='cuda:0')
0.9154515862464905


validation: 100%|█████████████████████████████████████████████████| 2797/2797 [06:18<00:00,  7.39img/s, loss (batch)=1]


...

validation: 100%|█████████████████████████████████████████████| 2797/2797 [05:50<00:00,  7.99img/s, loss (batch)=0.784]


Train: ./checkpoints/Vgg19SiamConc/checkpoint_epoch8.pth
tensor(0.6402, device='cuda:0')
0.7842162847518921


validation: 100%|█████████████████████████████████████████████| 2797/2797 [05:53<00:00,  7.91img/s, loss (batch)=0.842]


Train: ./checkpoints/Vgg19SiamConc/checkpoint_epoch9.pth
tensor(0.6176, device='cuda:0')
0.8423086404800415


PermissionError: [Errno 13] Permission denied: './checkpoints/Vgg19SiamConc/New folder'

In [14]:
model_parameters = filter(lambda p: p.requires_grad, net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [16]:
mem_params = sum([param.nelement()*param.element_size() for param in net.parameters()])
mem_bufs = sum([buf.nelement()*buf.element_size() for buf in net.buffers()])
mem = mem_params + mem_bufs

In [17]:
mem

149892372

In [ ]:

if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    #net = SiamUNetDiff()
    #net = SiamUNetConC()
    #net = SiameseUNetWithResnet50Encoder()
    #net = SiamUnet_diff_Full(3,5)
    #net = UNetWithResnet50Encoder(5,'custom','./checkpoints/Resnet50ContrastiveWeight/checkpoint_epoch0.pth')
    #net = UNet(out_classes=classes, up_sample_mode='conv_transpose',batch_norm=False)
    #net = SiameseUNet(n_channels=3, n_classes = classes, bilinear=bilinear)
    #net = UNetVgg19V2(out_classes=classes, up_sample_mode='conv_transpose')
    #net = vgg19bn_unet(5,True)
    #net = vgg19nobn_unet(5,True)
    #net = vgg19nobn_unetdouble(5,True)
    #net = VGGUnet19nobnspace(out_channels = 5, pretrained = True)
    #net = SiamUNetCombVgg19()
    net = SiamUNetConCVgg19()
    #net = resnet_unet()
    #net = VGG16Unet(out_channels = 5)
    #net = resnet_siamunet()
    #net = SiamUNetDiffVgg19()
    #net = SiamUNetFullConCVgg19()
    #net = SiamUNetConCResnet50(True,5)
    #net = UNETResnet50(True,2)
    '''net = smp.Unet(
        encoder_name='resnext101_32x8d',
        encoder_depth=5,
        encoder_weights='imagenet',
        decoder_use_batchnorm=False,
        decoder_channels=(1024,512,256,128, 64),
        decoder_attention_type=None,
        in_channels=3,
        classes=5,
        activation=None,
        aux_params=None
    )
    '''
    if loadstate:
        net.load_state_dict(torch.load(load, map_location=device))
        logging.info(f'Model loaded from {load}')

    net.to(device=device)
    train_net(net=net,
                  start_epoch = start_epoch,
                  epochs=epochs,
                  batch_size=batch_size,
                  learning_rate=lr,
                  device=device,
                  img_scale=scale,
                  train_percent = train,
                  val_percent=val,
                  save_checkpoint = save_checkpoint,
                  ampbool = ampbool,
                  traintype = traintype,
                  gradient_clipping=gradclip)

None
C:\Users\thanh/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth


validation: 100%|███████████████████████████████████████████████| 466/466 [00:56<00:00,  8.20img/s, loss (batch)=0.917]


tensor(0.4718, device='cuda:0')
[0, 0, 0, 0, 0]
0.9166298508644104
0.9219577716937183
0


validation: 100%|████████████████████████████████████████████████| 466/466 [00:56<00:00,  8.22img/s, loss (batch)=0.99]


tensor(0.5170, device='cuda:0')
[0, 0, 0, 0, 0]
0.990046501159668
0.9131598653305757
0


...

validation: 100%|████████████████████████████████████████████████| 466/466 [00:55<00:00,  8.45img/s, loss (batch)=1.25]


tensor(0.6396, device='cuda:0')
[0, 0, 0, 0, 0]
1.2479231357574463
0.2831485341348002
0


validation: 100%|████████████████████████████████████████████████| 466/466 [00:55<00:00,  8.34img/s, loss (batch)=1.45]


tensor(0.6248, device='cuda:0')
[0, 0, 0, 0, 0]
1.44805908203125
0.2842888791909229
0


Epoch 28/20:  68%|█████████████████████████████▊              | 1898/2797 [05:22<02:32,  5.91img/s, loss (batch)=0.618]